# Prototipo de prompting para comprender el uso de LLM's y su interacción.
## Trabajo Fin de Grado - Universidad de Burgos - Ingeniería Informática
### Autor: Fernando Pisot Serrano fps1001@alu.ubu.es

## Prompt simple
Se va a usar OpenAI 
Prompt: información que le damos al LLM. 
Contexto: es la 'memoria' del 

**Instalación de Bibliotecas**

Ejecuta la siguiente celda para instalar las bibliotecas necesarias.


In [ ]:
# %pip install openai # Para instalar OpenAI si no está instalado.
%pip install transformers torch # Para instalar Transformers y Torch si no están instalados.

https://www.youtube.com/watch?v=DELOjYAtbkg&t=977s 

**Cargar un Modelo y Generar Texto**

En esta sección, cargaremos un modelo de lenguaje y generaremos texto a partir de un prompt dado.

In [ ]:
from transformers import pipeline

# Cargar un pipeline de generación de texto
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B')

# Generar texto
prompt = "Once upon a time"
generated_text = generator(prompt, max_length=100, num_return_sequences=1)

print(generated_text[0]['generated_text'])
